In [1]:
import pandas as pd
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..')))


In [2]:
from src.feature_engineering import (
    split_title,
    extract_time_features,
    filter_ems_calls
)
from src.config import Config


df = pd.read_parquet("../data/cleaned_911.parquet")

df = split_title(df)
df = extract_time_features(df)



In [3]:
kpi_calls_per_hour = (
    df.groupby("hour")
      .size()
      .reset_index(name="call_count")
      .sort_values("hour")
)

kpi_calls_by_day = (
    df.groupby(["day_of_week", "day_name"])
      .size()
      .reset_index(name="call_count")
      .sort_values("day_of_week")
)

kpi_calls_by_department = (
    df.groupby("department")
      .size()
      .reset_index(name="call_count")
      .sort_values("call_count", ascending=False)
)

kpi_calls_by_month = (
    df.groupby(["month", "month_name"])
      .size()
      .reset_index(name="call_count")
      .sort_values("month")
)

kpi_top_zip_codes = (
    df.groupby("zip")
      .size()
      .reset_index(name="call_count")
      .dropna()
      .sort_values("call_count", ascending=False)
      .head(10)
)


In [4]:
config = Config()
engine = config.get_engine()

kpi_calls_per_hour.to_sql(
    "kpi_calls_per_hour",
    engine,
    if_exists="replace",
    index=False
)

kpi_calls_by_day.to_sql(
    "kpi_calls_by_day",
    engine,
    if_exists="replace",
    index=False
)

kpi_calls_by_department.to_sql(
    "kpi_calls_by_department",
    engine,
    if_exists="replace",
    index=False
)

kpi_calls_by_month.to_sql(
    "kpi_calls_by_month",
    engine,
    if_exists="replace",
    index=False
)

kpi_top_zip_codes.to_sql(
    "kpi_top_zip_codes",
    engine,
    if_exists="replace",
    index=False
)



10